### 영화 리뷰 감성분석 경진대회
데이콘 베이직 Basic | NLP | Accuracy

https://dacon.io/competitions/official/235864/overview/description

### 1. Import pacakges

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!pip install konlpy

In [2]:
from konlpy.tag import Okt
import re

In [3]:
okt = Okt()

### 2. Data loading
##### label (리뷰의 감성 종류) : 긍정(1), 부정(0)

In [4]:
train = pd.read_csv("train.csv")

print(train.shape)
train.head(2)

(5000, 3)


,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1


In [5]:
test = pd.read_csv("test.csv")

print(test.shape)
test.head(2)

(5000, 2)


,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!


In [6]:
submission = pd.read_csv("sample_submission.csv")

print(submission.shape)
submission.head(2)

(5000, 2)


,id,label
0,1,0
1,2,0


### 3. Preprocessing

#### 3-1) missing values

In [7]:
train.info()
train.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5000 non-null   int64 
 1   document  5000 non-null   object
 2   label     5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


id          0
document    0
label       0
dtype: int64

#### 3-2) regular expression 

In [10]:
train.head(1)
train["document"]
train.loc[2, "document"]

'괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~'

In [8]:
## Examples : 
txt_smpl = '괜123찮은 abc ZDBDase 음35악df영화가 또 나A왔E군요!!! 따뜻한 겨울이 될 것 같아요~'

### stype 1
re.sub("[괜찮은!~ ]", "", txt_smpl)
re.sub("[^1-9a-zA-Zㄱ-힣 ]", "", txt_smpl)

## stype 2
r = re.compile("[^1-9a-zA-Zㄱ-힣 ]")
re.sub(r, "", txt_smpl)

'괜123찮은 abc ZDBDase 음35악df영화가 또 나A왔E군요 따뜻한 겨울이 될 것 같아요'

In [12]:
def reg_exp(doc):
    
    return re.sub("[^1-9a-zA-Zㄱ-힣 ]", "", doc)

In [13]:
train["doc_re"] = train["document"].apply(reg_exp)

In [14]:
test["doc_re"] = test["document"].apply(reg_exp)

#### 3-3) stopwords
##### case 1

In [15]:
txt_smpl = "좋은 배우들로 3류영화를 찍은 안타까운 영화"

print(okt.nouns(txt_smpl))
print(okt.phrases(txt_smpl))
print(okt.morphs(txt_smpl, norm=True, stem=True))
print(okt.pos(txt_smpl, norm=True, stem=True))

['배우', '류', '영화', '영화']
['좋은 배우들', '3류영화', '안타까운 영화', '배우들', '영화']
['좋다', '배우', '들', '로', '3', '류', '영화', '를', '찍다', '안타깝다', '영화']
[('좋다', 'Adjective'), ('배우', 'Noun'), ('들', 'Suffix'), ('로', 'Josa'), ('3', 'Number'), ('류', 'Noun'), ('영화', 'Noun'), ('를', 'Josa'), ('찍다', 'Verb'), ('안타깝다', 'Adjective'), ('영화', 'Noun')]


In [24]:
stopwords = ["이","또","를","로"]

In [39]:
output = []
for txt in okt.morphs(txt_smpl, norm=True, stem=True):
    if txt not in stopwords:
#         print(txt)
        output.append(txt)
output        
" ".join(output)                

'좋다 배우 들 3 류 영화 찍다 안타깝다 영화'

In [16]:
def del_stopwords(doc):
    output = []
    for txt in okt.morphs(doc, norm=True, stem=True):
        if txt not in stopwords:
            output.append(txt)
            
    return " ".join(output)    

In [27]:
del_stopwords(txt_smpl)

'좋은 배우 들 3 류 영화 찍은 안타까운 영화'

In [ ]:
train["doc_re"].apply(del_stopwords)
test["doc_re"].apply(del_stopwords)

##### case 2

In [17]:
stop_pos = ["Suffix","Josa","KoreanParticle","Determiner"]

In [48]:
txt_smpl = "히치콕이이영화를봤다면분명박수를쳤을듯"
okt.pos(txt_smpl, norm=True, stem=True)

[('히치콕', 'Noun'),
 ('이', 'Determiner'),
 ('이영화', 'Noun'),
 ('를', 'Josa'),
 ('보다', 'Verb'),
 ('분명', 'Noun'),
 ('박수', 'Noun'),
 ('를', 'Josa'),
 ('치다', 'Verb')]

In [18]:
output = []
for txt in okt.pos(txt_smpl, norm=True, stem=True):
    if txt[1] not in stop_pos:
        output.append(txt[0])

print(txt_smpl)        
print(" ".join(output))

좋은 배우들로 3류영화를 찍은 안타까운 영화
좋다 배우 3 류 영화 찍다 안타깝다 영화


In [19]:
def del_pos(doc):
    output = []
    for txt in okt.pos(doc, norm=True, stem=True):
        if txt[1] not in stop_pos:
            output.append(txt[0])
            
    return " ".join(output)   

In [20]:
del_pos(txt_smpl)

'좋다 배우 3 류 영화 찍다 안타깝다 영화'

In [21]:
train['doc_pos'] = train["doc_re"].apply(del_pos)
test['doc_pos'] = test["doc_re"].apply(del_pos)

In [22]:
train.head(3)

,id,document,label,doc_re,doc_pos
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,영상 음악 이쁘다 해도 미화 시키다 불륜 일 뿐
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯,히치콕 이 영화 보다 분명 박수 치다
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1,괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요,괜찮다 음악 영화 또 나오다 따뜻하다 겨울 되다 것 같다


##### case 3

In [23]:
pick_pos = ["Noun","Adjective","Adverb","Verb","Exclamation"]#,"Conjunction"]

In [24]:
def find_pos(doc):
    output = []
    for txt in okt.pos(doc, norm=True, stem=True):
        if txt[1] in pick_pos:
            output.append(txt[0])
            
    return " ".join(output)   

In [25]:
find_pos(txt_smpl)

'좋다 배우 류 영화 찍다 안타깝다 영화'

In [26]:
train['pick_pos'] = train["doc_re"].apply(find_pos)
test['pick_pos'] = test["doc_re"].apply(find_pos)

In [27]:
train.tail(3)

,id,document,label,doc_re,doc_pos,pick_pos
4997,4998,가장 실망스러운 영화.. 지금까지 본영화중..,0,가장 실망스러운 영화 지금까지 본영화중,가장 실망 영화 지금 본 영화,가장 실망 영화 지금 영화
4998,4999,"이런 평점 테러, 네이버에서 좀 막아야 하는 것 아닌가?",1,이런 평점 테러 네이버에서 좀 막아야 하는 것 아닌가,이렇다 평점 테러 네이버 좀 막다 하다 것 아니다,이렇다 평점 테러 네이버 좀 막다 하다 것 아니다
4999,5000,"여주인공이 전작에서는 귀여웠는데, 여기선 완전 망했네, 실망이다",0,여주인공이 전작에서는 귀여웠는데 여기선 완전 망했네 실망이다,여 주인공 전작 귀엽다 여 기선 완전 망하다 실망,주인공 전작 귀엽다 기선 완전 망하다 실망


### 4. Modeling
#### 4-1) feature engineering

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

##### CountVectorizer

In [91]:
c_vect = CountVectorizer()

In [97]:
pd.concat([train["pick_pos"],test["pick_pos"]]).head(3)

0           영상 음악 이쁘다 해도 미화 시키다 불륜 뿐
1               히치콕 이 영화 보다 분명 박수 치다
2    괜찮다 음악 영화 또 나오다 따뜻하다 겨울 되다 것 같다
Name: pick_pos, dtype: object

In [88]:
list(pd.concat([train["pick_pos"],test["pick_pos"]]))

['영상 음악 이쁘다 해도 미화 시키다 불륜 뿐',
 '히치콕 이 영화 보다 분명 박수 치다',
 '괜찮다 음악 영화 또 나오다 따뜻하다 겨울 되다 것 같다',
 '아무래도 지금 보기 너무 유치하다',
 '지금 영화 그렇다 이 영화 역시 일본 대한 미화 여전하다',
 '인생 있다 손까락 꼽히다 영화 원작 능가 리메이크 작',
 '성룡 영화 최악 차라리 러시아워 훨씬 나다',
 '마음 성숙하다 보다 다르다 보이다 영화 걸작',
 '가슴 찡하다 오래 기억 남 영화',
 '결말 기분 나쁘다 공포 매니아 결말',
 '즐겁다 보다 성장 영화 화이팅',
 '긴장감 없다 재미 없다 주제 없다',
 '박신양 때문 프로젝트 망하다 윽 내 사랑',
 '말 필요없다 절대 길다 느껴지다 않다',
 '짧다 임팩트 있다 영화 문채원 연기 정말 좋다',
 '담담하다 서술 해내다 기적 같다 스포츠 사의 하다 이변',
 '우리 살다 세계 의심 하리',
 '재미 때 가리다 없다 내용 아무리 만화 라지 거지같다',
 '살짝 지루하다 수 있다 두 배우 명연기 소재 참신하다',
 '왜 전투씬 없다 액션 만화 전투씬 왜 없다 왜 적 다 발리다',
 '처절하다 영화 슬프다 영화 씁쓸하다 영화',
 '처음 끝 유치 하다 그 없다 난해하다 영화',
 '전쟁 아이 목소리 차다 거슬리다',
 '멋지다 상상력 아름답다 영화 여배우 소이 재 발견',
 '그 시대 이 영화 최고 이다',
 '재미없다 다애 싫어하다',
 '한글 이다 션 랜 다',
 '개 빡치다 하다 드라마 질퍽거려너 토나오다 지경',
 '카 메 안경 재밌다 지루하다',
 '닥터후 시리즈 뭐 관심',
 '정말 것 하다 일이 없다 보다 배속',
 '현실 지도 환상 지도 않다 불행하다 로맨틱 코미디',
 '소설 모욕 하다 영화 추상미 자살씬 기억 남다',
 '은 잊다 울컥',
 '왜 영점 없다 점도 아깝다',
 '좋다 의도 만들다 영화 내 그 의도 느끼다 없다',
 '눈물 흘리다 보다 그리스도인 추천 하다',
 '역시 피터 잭슨 완전 재밌다',
 '영화

In [30]:
c_vect.fit_transform(list(pd.concat([train["pick_pos"],test["pick_pos"]])))

<10000x8551 sparse matrix of type '<class 'numpy.int64'>'
	with 59101 stored elements in Compressed Sparse Row format>

In [36]:
c_vect.fit(list(pd.concat([train["pick_pos"],test["pick_pos"]])))

CountVectorizer()

In [37]:
X_train = c_vect.transform(train["pick_pos"])
y_train = train["label"]

In [ ]:
print(X_train.shape)
print(X_train)

In [43]:
X_test = c_vect.transform(test["pick_pos"])

In [46]:
test.head(3)

,id,document,doc_re,doc_pos,pick_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함,시간 때우다 좋다 영화 지루함,시간 때우다 좋다 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱
2,3,Childhood fantasy,Childhood fantasy,Childhood fantasy,


In [ ]:
print(X_test.shape)
print(X_test)

##### TF-IDF 

In [130]:
t_vect = TfidfVectorizer()

In [131]:
t_vect.fit(list(pd.concat([train["pick_pos"],test["pick_pos"]])))

TfidfVectorizer(analyzer='char')

In [132]:
X_train = t_vect.transform(train["pick_pos"])
y_train = train["label"]

In [ ]:
print(X_train.shape)
print(X_train)

In [97]:
X_test = t_vect.transform(test["pick_pos"])

In [99]:
test.head(1)

,id,document,doc_re,doc_pos,pick_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함,시간 때우다 좋다 영화 지루함,시간 때우다 좋다 영화 지루함


In [ ]:
print(X_test.shape)
print(X_test)

#### 4-2) Model : LogisticReg

In [47]:
from sklearn.linear_model import LogisticRegression

In [100]:
l_reg = LogisticRegression(class_weight="balanced", max_iter=3000, random_state=44)

In [49]:
l_reg.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=3000, random_state=44)

In [50]:
pred = l_reg.predict(X_test)
pred

array([0, 1, 0, ..., 1, 1, 1], dtype=int64)

#### 4-3) Model : RandomForest

In [64]:
from sklearn.ensemble import RandomForestClassifier

In [101]:
rf_clf = RandomForestClassifier(n_estimators=400, random_state=44)

In [66]:
rf_clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=800, random_state=44)

In [67]:
pred = rf_clf.predict(X_test)
pred

array([1, 1, 1, ..., 1, 0, 1], dtype=int64)

#### 4-4) Model : LightGBM

In [74]:
from lightgbm import LGBMClassifier

In [102]:
lgbm_clf = LGBMClassifier(random_state=44)

In [82]:
X_train = X_train.astype(np.float32)

In [83]:
lgbm_clf.fit(X_train, y_train)

LGBMClassifier(random_state=44)

#### 4-5) Model : Naive Bayes

In [108]:
from sklearn.naive_bayes import BernoulliNB

In [109]:
bnb_clf = BernoulliNB()

In [ ]:
bnb_clf.fit(X_train, y_train)

### 5. Submission / Report

In [52]:
submission["label"] = pred
submission.head(3)

,id,label
0,1,0
1,2,1
2,3,0


In [53]:
submission.to_csv("dacom_movie-220321_1348.csv", index=False)

### 6. Evaluation / Validation

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [133]:
X_fit, X_val, y_fit, y_val = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train, random_state=44)

In [125]:
print(X_train.shape)
print(X_fit.shape, X_val.shape)
print(y_fit.shape, y_val.shape)

(5000, 1346)
(4500, 1346) (500, 1346)
(4500,) (500,)


#### 6-1) CountVectorize, TF-IDF
##### LogisticReg

In [134]:
l_reg.fit(X_fit, y_fit)
pred_val = l_reg.predict(X_val)

accuracy_score(y_val, pred_val)

0.824

##### RandomForest

In [135]:
rf_clf.fit(X_fit, y_fit)
pred_val = rf_clf.predict(X_val)

accuracy_score(y_val, pred_val)

0.808

##### LightGBM

In [89]:
X_fit = X_fit.astype(np.float32)
X_val = X_val.astype(np.float32)

In [136]:
lgbm_clf.fit(X_fit, y_fit)
pred_val = lgbm_clf.predict(X_val)

accuracy_score(y_val, pred_val)

0.82

##### Naive Bayes

In [137]:
bnb_clf.fit(X_fit, y_fit)
pred_val = bnb_clf.predict(X_val)

accuracy_score(y_val, pred_val)

0.802